In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from evaluation import *
from filters import *
from utility import *
from features import *

## Öffnen von Hdf mittels pandas

In [2]:
hdf = pd.HDFStore('../../../data/raw/TestMessungen_NEU.hdf')

## Beispiel Erkenner

### datensätze vorbereiten

In [5]:
# generate datasets
tst = ['1','2','3']
tst_ds = []

for t in tst:

    df_tst = hdf.get('/x1/t'+t+'/trx_3_1')
    lst = df_tst.columns[df_tst.columns.str.contains('_ifft_')]
    
    #df_tst_cl,_ = distortion_filter(df_tst_cl)
    
    groups = get_trx_groups(df_tst)
    df_std = rf_grouped(df_tst, groups=groups, fn=rf_std_single, label='target')
    df_mean = rf_grouped(df_tst, groups=groups, fn=rf_mean_single)
    
    df_all = pd.concat( [df_std, df_mean], axis=1 )
    
    df_all = cf_std_window(df_all, window=4, label='target')
    
    df_tst_sum = generate_class_label_presence(df_all, state_variable='target')
    
    # remove index column
    df_tst_sum = df_tst_sum[df_tst_sum.columns.values[~df_tst_sum.columns.str.contains('index')].tolist()]
    print('Columns in Dataset:',t)
    print(df_tst_sum.columns)
    

    tst_ds.append(df_tst_sum.copy())
    

Columns in Dataset: 1
Index(['col_std_rf_std_trx_1_3_ifft_0', 'col_std_rf_std_trx_2_1_ifft_0',
       'col_std_rf_std_trx_2_3_ifft_0', 'col_std_rf_std_trx_3_1_ifft_0',
       'col_std_rf_std_trx_4_1_ifft_0', 'col_std_rf_std_trx_4_3_ifft_0',
       'col_std_rf_mean_trx_1_3_ifft_0', 'col_std_rf_mean_trx_2_1_ifft_0',
       'col_std_rf_mean_trx_2_3_ifft_0', 'col_std_rf_mean_trx_3_1_ifft_0',
       'col_std_rf_mean_trx_4_1_ifft_0', 'col_std_rf_mean_trx_4_3_ifft_0',
       'target'],
      dtype='object')
Columns in Dataset: 2
Index(['col_std_rf_std_trx_1_3_ifft_0', 'col_std_rf_std_trx_2_1_ifft_0',
       'col_std_rf_std_trx_2_3_ifft_0', 'col_std_rf_std_trx_3_1_ifft_0',
       'col_std_rf_std_trx_4_1_ifft_0', 'col_std_rf_std_trx_4_3_ifft_0',
       'col_std_rf_mean_trx_1_3_ifft_0', 'col_std_rf_mean_trx_2_1_ifft_0',
       'col_std_rf_mean_trx_2_3_ifft_0', 'col_std_rf_mean_trx_3_1_ifft_0',
       'col_std_rf_mean_trx_4_1_ifft_0', 'col_std_rf_mean_trx_4_3_ifft_0',
       'target'],
      dtyp

### validierung hold-out

In [6]:
# holdout validation
print(hold_out_val(tst_ds, target='target', include_self=False, cl='rf', verbose=False, random_state=1))

(0.66623012582470453, 0.079498802680287986)


### extra feature "peak to peak"

In [9]:
# generate datasets
tst = ['1','2','3']
tst_ds_2 = []

for t in tst:

    df_tst = hdf.get('/x1/t'+t+'/trx_3_1')
    lst = df_tst.columns[df_tst.columns.str.contains('_ifft_')]
    
    #df_tst_cl,_ = distortion_filter(df_tst_cl)
    
    df_tst = generate_class_label_presence(df_all, state_variable='target')
    
    groups = get_trx_groups(df_tst)
    df_std = rf_grouped(df_tst, groups=groups, fn=rf_std_single, label='target')
    df_mean = rf_grouped(df_tst, groups=groups, fn=rf_mean_single)
    df_p2p = rf_grouped(df_tst, groups=groups, fn=rf_ptp_single)
    
    df_all = pd.concat( [df_std, df_mean, df_p2p], axis=1 )
    
    df_all = cf_std_window(df_all, window=4, label='target')
    
    df_tst_sum = df_tst
    
    # remove index column
    df_tst_sum = df_tst_sum[df_tst_sum.columns.values[~df_tst_sum.columns.str.contains('index')].tolist()]
    print('Columns in Dataset:',t)
    print(df_tst_sum.columns)
    

    tst_ds_2.append(df_tst_sum.copy())
    

Columns in Dataset: 1
Index(['col_std_rf_std_trx_1_3_ifft_0', 'col_std_rf_std_trx_2_1_ifft_0',
       'col_std_rf_std_trx_2_3_ifft_0', 'col_std_rf_std_trx_3_1_ifft_0',
       'col_std_rf_std_trx_4_1_ifft_0', 'col_std_rf_std_trx_4_3_ifft_0',
       'col_std_rf_mean_trx_1_3_ifft_0', 'col_std_rf_mean_trx_2_1_ifft_0',
       'col_std_rf_mean_trx_2_3_ifft_0', 'col_std_rf_mean_trx_3_1_ifft_0',
       'col_std_rf_mean_trx_4_1_ifft_0', 'col_std_rf_mean_trx_4_3_ifft_0',
       'col_std_rf_ptp_trx_1_3_ifft_0', 'col_std_rf_ptp_trx_2_1_ifft_0',
       'col_std_rf_ptp_trx_2_3_ifft_0', 'col_std_rf_ptp_trx_3_1_ifft_0',
       'col_std_rf_ptp_trx_4_1_ifft_0', 'col_std_rf_ptp_trx_4_3_ifft_0',
       'target'],
      dtype='object')
Columns in Dataset: 2
Index(['col_std_rf_std_trx_1_3_ifft_0', 'col_std_rf_std_trx_2_1_ifft_0',
       'col_std_rf_std_trx_2_3_ifft_0', 'col_std_rf_std_trx_3_1_ifft_0',
       'col_std_rf_std_trx_4_1_ifft_0', 'col_std_rf_std_trx_4_3_ifft_0',
       'col_std_rf_mean_trx_1_3_if

In [10]:
# holdout validation
print(hold_out_val(tst_ds_2, target='target', include_self=False, cl='rf', verbose=False, random_state=1))

(0.61662994204003829, 0.13067575746487756)


## Schließen von HDF Store

In [ ]:
hdf.close()